# Modules

In [31]:
import tensorflow as tf

from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout, Input, Flatten, Dense, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [32]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

# Dataset preparation

In [33]:
batchsize = 16

In [34]:
# Training dataset

train_data_gen = ImageDataGenerator(rotation_range=0.2, 
                                        shear_range=0.2,
                                        zoom_range=0.2, 
                                        width_shift_range=0.2,
                                        height_shift_range=0.2, 
                                        rescale=1./255,
                                        validation_split=0.2)

train_data = train_data_gen.flow_from_directory('dataset/train', 
                                                    target_size=(80, 80), 
                                                    batch_size=batchsize, 
                                                    class_mode='categorical',
                                                    subset='training'
                                                    )

validation_data = train_data_gen.flow_from_directory('dataset/train', 
                                                    target_size=(80, 80), 
                                                    batch_size=batchsize, 
                                                    class_mode='categorical',
                                                    subset='validation'
                                                    )

Found 64719 images belonging to 2 classes.
Found 16179 images belonging to 2 classes.


In [35]:
# Testing dataset

test_data_gen = ImageDataGenerator(rescale=1./255)
test_data = test_data_gen.flow_from_directory('dataset/test', 
                                                    target_size=(80, 80), 
                                                    batch_size=batchsize, 
                                                    class_mode='categorical'
                                                    )

Found 4000 images belonging to 2 classes.


# Model preparation

In [36]:
# Get knowledge from InceptionV3 pretrained model, so we don't need to train the model from scratch
bmodel = InceptionV3(include_top=False, weights='imagenet', input_tensor=Input(shape=(80, 80, 3), batch_size=batchsize))
hmodel = bmodel.output
hmodel = Flatten()(hmodel)

# The only layers we need to train
# This is the layers' illustration https://www.google.com/url?sa=i&url=https%3A%2F%2Ftowardsdatascience.com%2Fthe-most-intuitive-and-easiest-guide-for-convolutional-neural-network-3607be47480&psig=AOvVaw0CYHQ2oruhWMmKK2Lxm5cN&ust=1668996857968000&source=images&cd=vfe&ved=0CBAQjRxqFwoTCLDJs9TYu_sCFQAAAAAdAAAAABAU
hmodel = Dense(64, activation='relu')(hmodel)
hmodel = Dropout(0.5)(hmodel)
hmodel = Dense(2, activation='softmax')(hmodel)

model = Model(inputs=bmodel.input, outputs=hmodel)

for layer in bmodel.layers:
    layer.trainable = False

print('done')

done


In [37]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(16, 80, 80, 3)]    0           []                               
                                                                                                  
 conv2d_282 (Conv2D)            (16, 39, 39, 32)     864         ['input_4[0][0]']                
                                                                                                  
 batch_normalization_282 (Batch  (16, 39, 39, 32)    96          ['conv2d_282[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 activation_282 (Activation)    (16, 39, 39, 32)     0           ['batch_normalization_282[0

                                                                                                  
 batch_normalization_292 (Batch  (16, 7, 7, 96)      288         ['conv2d_292[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 batch_normalization_293 (Batch  (16, 7, 7, 32)      96          ['conv2d_293[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 activation_287 (Activation)    (16, 7, 7, 64)       0           ['batch_normalization_287[0][0]']
                                                                                                  
 activation_289 (Activation)    (16, 7, 7, 64)       0           ['batch_normalization_289[0][0]']
          

                                                                                                  
 conv2d_302 (Conv2D)            (16, 7, 7, 48)       13824       ['mixed1[0][0]']                 
                                                                                                  
 conv2d_305 (Conv2D)            (16, 7, 7, 96)       55296       ['activation_304[0][0]']         
                                                                                                  
 batch_normalization_302 (Batch  (16, 7, 7, 48)      144         ['conv2d_302[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 batch_normalization_305 (Batch  (16, 7, 7, 96)      288         ['conv2d_305[0][0]']             
 Normalization)                                                                                   
          

                                                                  'activation_311[0][0]',         
                                                                  'max_pooling2d_14[0][0]']       
                                                                                                  
 conv2d_316 (Conv2D)            (16, 3, 3, 128)      98304       ['mixed3[0][0]']                 
                                                                                                  
 batch_normalization_316 (Batch  (16, 3, 3, 128)     384         ['conv2d_316[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 activation_316 (Activation)    (16, 3, 3, 128)      0           ['batch_normalization_316[0][0]']
                                                                                                  
 conv2d_31

 batch_normalization_326 (Batch  (16, 3, 3, 160)     480         ['conv2d_326[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 activation_326 (Activation)    (16, 3, 3, 160)      0           ['batch_normalization_326[0][0]']
                                                                                                  
 conv2d_327 (Conv2D)            (16, 3, 3, 160)      179200      ['activation_326[0][0]']         
                                                                                                  
 batch_normalization_327 (Batch  (16, 3, 3, 160)     480         ['conv2d_327[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 activatio

 conv2d_337 (Conv2D)            (16, 3, 3, 160)      179200      ['activation_336[0][0]']         
                                                                                                  
 batch_normalization_337 (Batch  (16, 3, 3, 160)     480         ['conv2d_337[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 activation_337 (Activation)    (16, 3, 3, 160)      0           ['batch_normalization_337[0][0]']
                                                                                                  
 conv2d_333 (Conv2D)            (16, 3, 3, 160)      122880      ['mixed5[0][0]']                 
                                                                                                  
 conv2d_338 (Conv2D)            (16, 3, 3, 160)      179200      ['activation_337[0][0]']         
          

 activation_347 (Activation)    (16, 3, 3, 192)      0           ['batch_normalization_347[0][0]']
                                                                                                  
 conv2d_343 (Conv2D)            (16, 3, 3, 192)      147456      ['mixed6[0][0]']                 
                                                                                                  
 conv2d_348 (Conv2D)            (16, 3, 3, 192)      258048      ['activation_347[0][0]']         
                                                                                                  
 batch_normalization_343 (Batch  (16, 3, 3, 192)     576         ['conv2d_343[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 batch_normalization_348 (Batch  (16, 3, 3, 192)     576         ['conv2d_348[0][0]']             
 Normaliza

                                                                                                  
 batch_normalization_352 (Batch  (16, 3, 3, 192)     576         ['conv2d_352[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 batch_normalization_356 (Batch  (16, 3, 3, 192)     576         ['conv2d_356[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 activation_352 (Activation)    (16, 3, 3, 192)      0           ['batch_normalization_352[0][0]']
                                                                                                  
 activation_356 (Activation)    (16, 3, 3, 192)      0           ['batch_normalization_356[0][0]']
          

 activation_360 (Activation)    (16, 1, 1, 384)      0           ['batch_normalization_360[0][0]']
                                                                                                  
 activation_361 (Activation)    (16, 1, 1, 384)      0           ['batch_normalization_361[0][0]']
                                                                                                  
 activation_364 (Activation)    (16, 1, 1, 384)      0           ['batch_normalization_364[0][0]']
                                                                                                  
 activation_365 (Activation)    (16, 1, 1, 384)      0           ['batch_normalization_365[0][0]']
                                                                                                  
 batch_normalization_366 (Batch  (16, 1, 1, 192)     576         ['conv2d_366[0][0]']             
 Normalization)                                                                                   
          

 activation_374 (Activation)    (16, 1, 1, 384)      0           ['batch_normalization_374[0][0]']
                                                                                                  
 batch_normalization_375 (Batch  (16, 1, 1, 192)     576         ['conv2d_375[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 activation_367 (Activation)    (16, 1, 1, 320)      0           ['batch_normalization_367[0][0]']
                                                                                                  
 mixed9_1 (Concatenate)         (16, 1, 1, 768)      0           ['activation_369[0][0]',         
                                                                  'activation_370[0][0]']         
                                                                                                  
 concatena

# Train the model

In [28]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

In [38]:
checkpoint = ModelCheckpoint('models', 
                             monitor='val_loss', 
                             save_best_only=True, 
                             verbose=3)

# patience = max num of epochs when accuracy is not increasing
earlystop = EarlyStopping(monitor='val_loss', patience=7, verbose=3, restore_best_weights=True)

learning_rate = ReduceLROnPlateau(monitor='val_loss', patience=3, verbose=3)

callbacks = [checkpoint, earlystop, learning_rate]

In [39]:
try:
    model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

    model.fit(train_data,
          steps_per_epoch=train_data.samples//batchsize,
          validation_data=validation_data,
          validation_steps=validation_data.samples//batchsize,
          callbacks=callbacks,
          epochs=5)
except Exception as e:
    print(e)

C:\Users\felix\AppData\Local\Temp\ipykernel_14000\742531018.py:4: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_data,


Epoch 1/5
3826/4044 [===========================>..] - ETA: 42s - loss: 0.2062 - accuracy: 0.9183Graph execution error:

Detected at node 'model_3/dropout_3/dropout/Mul_1' defined at (most recent call last):
    File "C:\Users\felix\anaconda3\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\felix\anaconda3\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "C:\Users\felix\anaconda3\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\Users\felix\anaconda3\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
      app.start()
    File "C:\Users\felix\anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 712, in start
      self.io_loop.start()
    File "C:\Users\felix\anaconda3\lib\site-packages\tornado\platform\asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "C:\Users\felix\anaco

# Model evaluation

In [ ]:
acc_training, loss_training = model.evaluate_generator(train_data)
print(acc_training)
print(loss_training)

In [ ]:
acc_val, loss_val = model.evaluate_generator(validation_dataatadatioacc_vala)
print(acc_val)
print(loss_val)

In [ ]:
acc_test, loss_test = model.evaluate_generator(test_data)
print(acc_test)
print(loss_test)